In [10]:
!pip install meshio polyscope

In [11]:
import os
from collections import Counter

import meshio
import numpy as np
import polyscope as ps

In [12]:
mesh = meshio.read(os.path.join(os.getcwd(), '../meshes/Cube.001.msh'))

In [15]:
# Check if the mesh contains tetrahedral cells
if "tetra" in mesh.cells_dict:
    tetrahedra = mesh.cells_dict["tetra"]

    # Extract all faces from tetrahedra
    all_faces = []
    for tet in tetrahedra:
        # Each tetrahedron has 4 faces
        all_faces.extend([
            tuple(sorted([tet[0], tet[1], tet[2]])),  # Face 1
            tuple(sorted([tet[0], tet[1], tet[3]])),  # Face 2
            tuple(sorted([tet[0], tet[2], tet[3]])),  # Face 3
            tuple(sorted([tet[1], tet[2], tet[3]])),  # Face 4
        ])

    # Count the occurrences of each face
    face_counts = Counter(all_faces)

    # Surface faces are those that appear only once
    surface_faces = [face for face, count in face_counts.items() if count == 1]

    print(f"Total surface faces: {len(surface_faces)}")
    # Optionally, print the first few surface faces to verify
    print("First 10 Surface faces:", surface_faces[:10])

    # Initialize Polyscope
    ps.init()

    # Extract vertex positions
    vertices = mesh.points

    # Convert surface_faces to a NumPy array
    surface_faces_array = np.array(surface_faces)

    # Register the surface mesh with Polyscope
    ps_mesh = ps.register_surface_mesh("Surface Mesh", vertices, surface_faces_array)

    # (Optional) Customize the appearance
    ps_mesh.set_color([0.7, 0.7, 0.7])  # Set mesh color to light gray

    # Show the Polyscope interface
    ps.show()
else:
    print("The mesh does not contain tetrahedral cells.")

Total surface faces: 1514
First 10 Surface faces: [(9, 635, 712), (9, 635, 744), (16, 642, 722), (140, 164, 166), (61, 62, 65), (52, 53, 420), (269, 290, 291), (296, 299, 309), (230, 231, 234), (143, 169, 758)]
